### Basic statistics and CED 

In this notebook, we explore basic analysis of the positions and speeds to detect similarities and differences between behaviors. In particular, we compute the means and standard deviations on all output variables of navground simulations, and analyse their evolutions. In addition, we inspect also the Collisions, Efficacy and Deadlocks (CED).

We start importing some libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

from navground import core, sim
from navground.sim.ui.video import display_video_from_run

from perdiver.navground_io import parser, run_navground

plots_dir = os.path.join("plots", "basic_metrics")
experiment_dir = "experiments"
os.makedirs(plots_dir, exist_ok=True)
os.makedirs("experiments", exist_ok=True)

In [ ]:
args = parser.parse_args([
        '--scenario', 'CrossTorus',
        '--side', '6.5',
        '--length', '15.0',
        '--width', '3.5',
        '--num_runs', '15',
        '--num_steps', '500',
        '--time_step', '0.1',
        '--num_agents', '12',
        '--max_speed', '1.66',
        '--optimal_speed_min', '0.1',
        '--optimal_speed_min', '0.15',
        '--radius', '0.4',
        '--safety_margin', '0.1',
        '--epsilon', '20',
        '--time_delay', '10',
])

behavior_list = ["ORCA", "HL", "HRVO", "SocialForce"]
marker_behavior = {"ORCA": "o", "HL": "X", "HRVO": "+", "Dummy": "*", "SocialForce": "x"}
color_behavior = {}
for i, behavior in enumerate(behavior_list):
    color_behavior[behavior] = mpl.colormaps["Set1"](i / (len(behavior_list) +1))

runs = {}
for behavior in behavior_list:
    args.behavior = behavior
    runs[behavior] = run_navground(args)

Visualise runs.

In [ ]:
display_video_from_run(run=runs["ORCA"][0], factor=6.0, fps=20)

Obtain statistic measures related to both poses and twists from navground simulations.

In [ ]:
args.timestep_list = list(range(30, 100, 5))
variables_behavior = {}
for j, behavior in enumerate(behavior_list):
    variables_list = []
    for ridx in range(args.num_runs):
        run = runs[behavior][ridx]
        variables_run = []
        ps = np.array(run.poses)
        twists = np.array(run.twists)
        for idx, step in enumerate(args.timestep_list):
            if step >= run.recorded_steps:
                variables_run.append(variables_run[-1])
                continue
            X = ps[step]
            vel_X = twists[step]
            variables_run.append(np.hstack((np.mean(X, axis=0), np.std(X, axis=0), np.mean(vel_X, axis=0), np.std(vel_X, axis=0))))
        # end for 
        
        variables_list.append(np.array(variables_run).transpose())
    # end for
    variables_behavior[behavior] = variables_list

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=6, figsize=(12,8))
labels = np.array([
    ["x mean", "y mean", "angle mean", "x std", "y std", "angle std"],
    ["v_x mean", "v_y mean", "angle speed mean", "v_x std", "v_y std", "angular speed std"]
]).transpose()
for irow in range(labels.shape[0]):
    for icol in range(labels.shape[1]):
        for behavior in behavior_list:
            for variables_run in variables_behavior[behavior]:
                ax[irow, icol].set_title(labels[irow, icol])
                timesteps = args.timestep_list[:len(variables_run[icol*6 + irow])]
                ax[irow, icol].plot(timesteps, variables_run[icol*6 + irow], color=color_behavior[behavior], label=behavior)

plt.tight_layout()
# Remove duplicate labels 
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys())
plt.savefig(os.sep.join((plots_dir, "basic_metrics_signals.png")))

Velocity in $x$ and $y$ standard deviations are very good at seeing the differences. Let us repeat the plot for velocities only.

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=3, figsize=(8,4), squeeze=False)
labels = np.array([["v_x std", "v_y std", "angular speed std"]]).transpose()
for irow in range(labels.shape[0]):
    for icol in range(labels.shape[1]):
        for behavior in behavior_list:
            for variables_run in variables_behavior[behavior]:
                ax[irow, icol].set_title(labels[irow, icol])
                variables = variables_run[9 + irow]
                timesteps = args.timestep_list[:len(variables)]
                ax[irow, icol].plot(timesteps, variables, color=color_behavior[behavior], label=behavior)


# Remove duplicate labels 
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "basic_metrics_velocities.png")))

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(12,4))
labels = np.array([
    ["x mean", "y mean", "angle mean", "x std", "y std", "angule std"],
    ["v_x mean", "v_y mean", "angle speed mean", "v_x std", "v_y std", "angular speed std"]
])
variables = np.vstack([variables_behavior[behavior] for behavior in behavior_list])
for irow in range(labels.shape[0]):
    for icol in range(labels.shape[1]):
        ax[irow, icol].set_title(labels[irow, icol])
        pca = PCA(n_components=2)
        Y = pca.fit_transform(variables[:, irow*6 + icol, :])
        Y_dict = {}
        for i, behavior in enumerate(behavior_list):
            Y_dict[behavior] = Y[args.num_runs*i:args.num_runs*(i+1)]
        # end for
        for behavior in behavior_list:
            ax[irow, icol].scatter(Y_dict[behavior][:args.num_runs][:,0], Y_dict[behavior][:args.num_runs][:,1], color=color_behavior[behavior], marker=marker_behavior[behavior], label=behavior)
# Remove duplicate labels 
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "basic_metrics_PCA.png")))

Repeat PCA only for standard deviations of $v_x$ and $v_y$

In [ ]:
fig, ax = plt.subplots(ncols=3, nrows=1, figsize=(9,3), squeeze=False)
labels = np.array([
    [ "v_x std", "v_y std", "angular speed std"]
])
variables = np.vstack([variables_behavior[behavior] for behavior in behavior_list])
for irow in range(labels.shape[0]):
    for icol in range(labels.shape[1]):
        ax[irow, icol].set_title(f"PCA {labels[irow, icol]}")
        pca = PCA(n_components=2)
        Y = pca.fit_transform(variables[:, 9 + icol, :])
        Y_dict = {}
        for i, behavior in enumerate(behavior_list):
            Y_dict[behavior] = Y[args.num_runs*i:args.num_runs*(i+1)]
        # end for
        for behavior in behavior_list:
            ax[irow, icol].scatter(Y_dict[behavior][:args.num_runs][:,0], Y_dict[behavior][:args.num_runs][:,1], color=color_behavior[behavior], marker=marker_behavior[behavior], label=behavior)

# Remove duplicate labels 
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "std_velocities_PCA.png")))

Let us see how these simulations relate to collisions, efficacy and deadlocks (CED).

In [ ]:
from perdiver.navground_io import extract_data

initial_step = 0
final_step = args.num_steps
deadlocks = {}
collisions = {}
efficacy = {}
for behavior in behavior_list:
    df = extract_data(runs[behavior], initial_step, final_step)
    collisions[behavior] =  list(df.collisions)
    deadlocks[behavior] = list(df.deadlocks)
    efficacy[behavior] = list(df.efficacy)

In [ ]:
fig, ax = plt.subplots(figsize=(9,3), ncols=3)
for behavior in behavior_list:
    ax[0].scatter(deadlocks[behavior], collisions[behavior], color=color_behavior[behavior], label=behavior, marker=marker_behavior[behavior])
    ax[0].set_xlabel("deadlocks")
    ax[0].set_ylabel("collisions")
    ax[1].scatter(efficacy[behavior], collisions[behavior], color=color_behavior[behavior], label=behavior, marker=marker_behavior[behavior])
    ax[1].set_xlabel("efficacy")
    ax[1].set_ylabel("collisions")
    ax[2].scatter(efficacy[behavior], deadlocks[behavior], color=color_behavior[behavior], label=behavior, marker=marker_behavior[behavior])
    ax[2].set_xlabel("efficacy")
    ax[2].set_ylabel("deadlocks")
# end for
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc=(0.3,0),  ncol=len(behavior_list))
plt.tight_layout()
plt.savefig(os.sep.join((plots_dir, "efficacy_collisions_cross_torus.png")))